In [31]:
import pickle
import itertools
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import ranksums

In [34]:
pickle_in = open("errors.p","rb")
errors = pickle.load(pickle_in)


In [36]:
p_vals = []
perms = itertools.combinations(errors.keys(), r=2)
for perm in perms:
    a1 = errors[perm[0]].astype(np.float)
    a2 = errors[perm[1]].astype(np.float)
    t_stat, p_val = ttest_ind(a1,a2)
    p_vals.append(p_val)
    print(perm[0] +'-' + perm[1],t_stat,p_val )

Lasso-LassoLars -0.1851344254977115 0.8532799572802177
Lasso-LogisticRegression 0.43289606125573965 0.6654790495967557
Lasso-PassiveAggressiveClassifier -0.23813787127094255 0.811977352113328
Lasso-RidgeRegression -0.009317187098025361 0.9925738078939808
LassoLars-LogisticRegression 0.6368423876509429 0.5248344770090425
LassoLars-PassiveAggressiveClassifier -0.07520884381232948 0.9401111993427456
LassoLars-RidgeRegression 0.17516663808131008 0.8610963876865952
LogisticRegression-PassiveAggressiveClassifier -0.6246577273000408 0.5327891122369554
LogisticRegression-RidgeRegression -0.4411892626637381 0.6594731767196389
PassiveAggressiveClassifier-RidgeRegression 0.22930291675275183 0.8188287764555359


In [32]:
perms = itertools.combinations(errors.keys(), r=2)
for perm in perms:
    x = errors[perm[0]]
    y = errors[perm[1]].astype(np.float)
    
    t_stat, p_val = ranksums(x,y)
    print(perm[0] +'-' + perm[1],t_stat,p_val )

Lasso-LassoLars 1.039485766758494 0.29857887423105844
Lasso-LogisticRegression 0.41414141414141414 0.6787705444537818
Lasso-PassiveAggressiveClassifier -0.10560146923783287 0.9158985589882459
Lasso-RidgeRegression 1.2350780532598715 0.21680145643800885
LassoLars-LogisticRegression -0.317722681359045 0.750695304809174
LassoLars-PassiveAggressiveClassifier -0.7741046831955923 0.43886888825104686
LassoLars-RidgeRegression 0.18640955004591367 0.8521236038341486
LogisticRegression-PassiveAggressiveClassifier -0.5711662075298439 0.5678869824552863
LogisticRegression-RidgeRegression 0.559228650137741 0.5760056825391491
PassiveAggressiveClassifier-RidgeRegression 0.9963269054178145 0.3190913351481137


In [38]:
from statsmodels.sandbox.stats.multicomp import multipletests

In [63]:
reject, p_adjusted, alphacSidak, alphacBonf = multipletests(p_vals, alpha=0.05, method='bonferroni')

In [87]:
p_adjusted

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [60]:
p_adjusted[2]

0.005116196891823743

In [50]:
from mleap.analyze_results import AnalyseResults
from mleap.data import Data

In [52]:
data = Data()
input_io = data.open_hdf5('data/delgado.hdf5', mode='r')
out_io = data.open_hdf5('data/experiments.hdf5', mode='a')
analyze = AnalyseResults(hdf5_output_io=out_io, hdf5_input_io=input_io)

In [54]:
t_test, df_t = analyze.t_test(errors)

In [68]:
df_t

,pair,t_statistic,p_value
0,Lasso - LassoLars,-0.185134,0.853280
1,Lasso - LogisticRegression,0.432896,0.665479
2,Lasso - PassiveAggressiveClassifier,-0.238138,0.811977
3,Lasso - RidgeRegression,-0.009317,0.992574
4,LassoLars - LogisticRegression,0.636842,0.524834
5,LassoLars - PassiveAggressiveClassifier,-0.075209,0.940111
6,LassoLars - RidgeRegression,0.175167,0.861096
7,LogisticRegression - PassiveAggressiveClassifier,-0.624658,0.532789
8,LogisticRegression - RidgeRegression,-0.441189,0.659473
9,PassiveAggressiveClassifier - RidgeRegression,0.229303,0.818829


In [78]:
import pandas as pd
new_df = pd.DataFrame(df_t['pair'],df_t['p_value'], axis=0)

TypeError: __init__() got an unexpected keyword argument 'axis'

In [91]:
new_df = pd.concat( [df_t['pair'],pd.Series(p_adjusted, name='p_val') ], axis=1)

In [93]:
np.array(new_df)

array([['Lasso - LassoLars', 1.0],
       ['Lasso - LogisticRegression', 1.0],
       ['Lasso - PassiveAggressiveClassifier', 1.0],
       ['Lasso - RidgeRegression', 1.0],
       ['LassoLars - LogisticRegression', 1.0],
       ['LassoLars - PassiveAggressiveClassifier', 1.0],
       ['LassoLars - RidgeRegression', 1.0],
       ['LogisticRegression - PassiveAggressiveClassifier', 1.0],
       ['LogisticRegression - RidgeRegression', 1.0],
       ['PassiveAggressiveClassifier - RidgeRegression', 1.0]],
      dtype=object)